In [2]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [3]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = (4, 5)

rows = 3
cols = 3
total_params = rows*cols
time_intervals = 690
train_size = 0.8
lookback_window = 50

In [4]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

No earthquakes were recorded in this cell.


In [5]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [6]:
in_params = rows*cols*lookback_window
hidden1 = 40
hidden2 = 70
out_params = rows*cols

In [7]:
model = ANN(in_params, hidden1, hidden2, out_params)

if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor[:100])
# print(out)                    

In [8]:
# SETTINGS T1
pos_weight = torch.tensor([1])
pos_weight = pos_weight.cuda()
                          
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
n_epochs = 100

In [11]:
optimizer = optim.Adagrad(model.parameters(), lr=0.0001)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=10, lookback_window=lookback_window)

 10%|█         | 66/640 [00:00<00:00, 652.49it/s]

Epoch 1 completed. 
Training Loss: 0.551
Test Loss: 0.483


 10%|█         | 67/640 [00:00<00:00, 666.34it/s]

Epoch 2 completed. 
Training Loss: 0.543
Test Loss: 0.475


  8%|▊         | 49/640 [00:00<00:01, 489.77it/s]

...

Epoch 8 completed. 
Training Loss: 0.533
Test Loss: 0.457


  9%|▉         | 56/640 [00:00<00:01, 554.14it/s]

Epoch 9 completed. 
Training Loss: 0.532
Test Loss: 0.456


100%|██████████| 640/640 [00:01<00:00, 613.14it/s]

Epoch 10 completed. 
Training Loss: 0.531
Test Loss: 0.455


In [12]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 640/640 [00:00<00:00, 967.38it/s]


Evaluation results for training dataset:
4518 evaluations in total.
2313/2685 cells with earthquake activity were correctly identified. (86.15 %)
1367/1833 cells with no earthquake activity were correctly identified. (74.58 %)
466/2779 positives turned out to be false.
372/1739 negatives turned out to be false.
Preicision: 0.832
Recall: 0.861
Overall accuracy: 81.45 %
Evaluation results for training dataset:
1242 evaluations in total.
926/1026 cells with earthquake activity were correctly identified. (90.25 %)
176/216 cells with no earthquake activity were correctly identified. (81.48 %)
40/966 positives turned out to be false.
100/276 negatives turned out to be false.
Preicision: 0.959
Recall: 0.903
Overall accuracy: 88.73 %
